In [1]:
import torch
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)

/mnt/shared_home/vlialin/miniconda3/envs/peft_comparison/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-3b", load_in_4bit=True, device_map="auto")

# model.add_adapter("adapter", config="pfeiffer", set_active=True)
# model.train_adapter("adapter")

TypeError: T5ForConditionalGeneration.__init__() got an unexpected keyword argument 'load_in_4bit'

In [9]:
for name, module in model.named_modules():
    if "adapter" in name:
        module.to(device=model.device)#, dtype=torch.float32)

In [3]:
# print all model dtypes
for name, param in model.named_parameters():
    print(name, param.dtype)


shared.weight torch.float16
encoder.block.0.layer.0.SelfAttention.q.weight torch.int8
encoder.block.0.layer.0.SelfAttention.k.weight torch.int8
encoder.block.0.layer.0.SelfAttention.v.weight torch.int8
encoder.block.0.layer.0.SelfAttention.o.weight torch.int8
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight torch.float16
encoder.block.0.layer.0.layer_norm.weight torch.float16
encoder.block.0.layer.1.DenseReluDense.wi.weight torch.int8
encoder.block.0.layer.1.DenseReluDense.wo.weight torch.float32
encoder.block.0.layer.1.layer_norm.weight torch.float16
encoder.block.0.layer.1.adapters.adapter.adapter_down.0.weight torch.float32
encoder.block.0.layer.1.adapters.adapter.adapter_down.0.bias torch.float32
encoder.block.0.layer.1.adapters.adapter.adapter_up.weight torch.float32
encoder.block.0.layer.1.adapters.adapter.adapter_up.bias torch.float32
encoder.block.1.layer.0.SelfAttention.q.weight torch.int8
encoder.block.1.layer.0.SelfAttention.k.weight torch.int8
encoder.bl

In [10]:
input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids
out = model(input_ids, decoder_input_ids=input_ids, labels=input_ids)

In [13]:
out = model.generate(input_ids, num_beams=5)

RuntimeError: "log_softmax_lastdim_kernel_impl" not implemented for 'Half'